In [ ]:
from fastai.vision.all import *
from google.colab import drive

# Monteer Google Drive
drive.mount('/content/drive', force_remount=True)  # Zorg ervoor dat Drive is gemonteerd

# Definieer het pad naar je afbeeldingen
data_dir = Path('/content/drive/My Drive/Colab Notebooks/Screenshots/')  # Hoofdpad naar de afbeeldingen

# DataBlock voor het verwerken van afbeeldingen
dblock = DataBlock(
    blocks=(ImageBlock, CategoryBlock),  # We classificeren afbeeldingen
    get_items=get_image_files,           # Alle afbeeldingsbestanden ophalen
    splitter=RandomSplitter(valid_pct=0.2, seed=123),  # 20% voor validatie
    get_y=parent_label,                  # Labels worden gebaseerd op mappenamen
    item_tfms=Resize(180),               # Afbeeldingen worden geschaald naar 180x180
    batch_tfms=aug_transforms(size=180)  # Basisaugmentatie en schaling
)

# Maak een DataLoaders object
dls = dblock.dataloaders(data_dir, bs=64)

# Controleer de inhoud van de DataLoaders (bijvoorbeeld labels)
print(dls.vocab)  # Om te zien welke categorieën hij heeft herkend
dls.train.show_batch(max_n=4, nrows=1)
dls.valid.show_batch(max_n=4, nrows=1)

# Het model trainen met een vooraf getrainde ResNet-architectuur
learn = vision_learner(dls, resnet50, metrics=error_rate).to_fp16()

# Zoek naar de optimale learning rate
learn.lr_find()

#Discriminative Learning rates - Slice
learn.fit_one_cycle(3, 3e-3)
learn.unfreeze()

# Zoek naar de optimale learning rate
learn.lr_find()

# Gebruik de learning rate gevonden door lr_find()
learn.fine_tune(1, base_lr=1e-3)

# Model evaluatie en visualisatie
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(figsize=(12,12), dpi=60)
interp.plot_top_losses(5, nrows=5)

# Optionele callback voor vroegtijdig stoppen
# early_stopping_cb = EarlyStoppingCallback(monitor='valid_loss', patience=5)

# Train het model verder met precies 25 epochs
# learn.fit_one_cycle(25, cbs=[early_stopping_cb])
learn.fit_one_cycle(25, lr_max=slice(1e-6, 1e-4))

# Bewaar het getrainde model als een .pkl bestand
learn.export('/content/drive/My Drive/Colab Notebooks/ad-modelh7.pkl')  # Bewaar model naar Google Drive